## read detection results

In [6]:
import csv

In [75]:
class BBox(object):
    def __init__(self, startX, startY, endX, endY, confidence):
        self.confidence = confidence
        self.startX = startX
        self.startY = startY
        self.endX = endX
        self.endY = endY
        self.width = self.endX-self.startX
        self.height = self.endY-self.startY
        self.match = False
        
    def getWidth(self):
        
        return self.width
    def getHeight(self):
       
        return self.height
    def getArea(self):
        return self.width*self.height
    def getRec(self):
        return np.array([self.startX,self.startY,self.width,self.height])
    def setMatch(self):
        self.match = True
        
        
frame_num = 548


frameBBoxList = [[] for y in range(frame_num)] 


In [76]:
channel_num = 7
channel_list = [2,4,6]
method = 'faster-rcnn2'  #results/yolo, results_new/faster-rcnn2, results_new/mask-rcnn
dataset = 'Pathway1_1'
start_frame = 310
image_num = 550

filename = method+"_"+dataset+".txt"
TH=0.5

bbox_count = [[0 for x in range(image_num)] for y in range(channel_num)]
ppl_count = [[0 for x in range(image_num)] for y in range(channel_num)]
confidence_mat = [[0 for x in range(image_num)] for y in range(channel_num)]
area_mat = [[0 for x in range(image_num)] for y in range(channel_num)]

tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]
hatch_style = ['/',  '.', 'o', 'x', '|',  '+' ,   '-' ]
               
for i in range(len(tableau20)):
    r, g, b = tableau20[i]
    tableau20[i] = (r / 255., g / 255., b / 255.)

In [77]:

with open(filename, 'rb') as csvfile:
    
# 讀取 CSV 檔案內容
    rows = csv.reader(csvfile)
# 以迴圈輸出每一列
    for row in rows:    
        
        channel_idx = int(row[0])        
        if(channel_idx ==6):
            img_idx = int(row[1])
            bbox_count[channel_idx][img_idx] = bbox_count[channel_idx][img_idx]+1

            confidence = float(row[6])
            startX = int(float(row[2]))
            startY = int(float(row[3]))
            endX = int(float(row[4]))
            endY = int(float(row[5]))
            #already detected one face            
             
                
            
            frameBBoxList[img_idx].append(BBox(startX,startY,endX,endY,confidence))


csvfile.close()

In [12]:
for i in range(0,frame_num):
    print(i)
    if frameBBoxList[i]:
        print(frameBBoxList[i][0].startX)
        print(frameBBoxList[i][0].endX)
        print(frameBBoxList[i][0].startY)
        print(frameBBoxList[i][0].confidence)        
        print(frameBBoxList[i][0].endY)

        print(frameBBoxList[i][0].getWidth())
        print(frameBBoxList[i][0].getHeight())
        print(frameBBoxList[i][0].getArea())


    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
10
258
234
0.8096887
473
248
239
59272
106
8
319
255
0.93571496
471
311
216
67176
107
5
351
77
0.81501144
479
346
402
139092
108
51
323
189
0.872753
476
272
287
78064
109
96
321
213
0.9114705
472
225
259
58275
110
111
112
113
179
403
156
0.89289784
479
224
323
72352
114
207
406
147
0.9442491
478
199
331
65869
115
230
416
141
0.94998413
479
186
338
62868
116
251
435
146
0.9921124
479
184
333
61272
117
255
443
140
0.9903274
471
188
331
62228
118
266
449
149
0.9715485
462
183
313
57279
119
120
309
451
148
0.9587007
479
142
331
47002
121
312
465
141
0.9098763
479
153
338
51714
122
324
471
139
0.914964
452
147
313
46011
123
124
342
469
146
0.95465183
475
127
329
41783
125
330
469
12

528
529
530
531
532
533
534
535
40
307
161
0.8183463
479
267
318
84906
536
22
274
149
0.8066565
479
252
330
83160
537
538
539
540
7
295
63
0.84994286
479
288
416
119808
541
0
294
208
0.94243866
476
294
268
78792
542
4
258
54
0.81102055
479
254
425
107950
543
2
245
293
0.99251807
474
243
181
43983
544
0
241
282
0.9369677
467
241
185
44585
545
546
547


## tracking

In [42]:
import numpy as np
import os
import sys
import time
import argparse
import yaml, json
from PIL import Image

import matplotlib.pyplot as plt

import torch
import torch.utils.data as data
import torch.optim as optim

sys.path.insert(0, '.')
from modules.model import MDNet, BCELoss, set_optimizer
from modules.sample_generator import SampleGenerator
from modules.utils import overlap_ratio
from tracking.data_prov import RegionExtractor
from tracking.bbreg import BBRegressor
from tracking.gen_config import gen_config

opts = yaml.safe_load(open('tracking/options.yaml','r'))


In [43]:
def forward_samples(model, image, samples, out_layer='conv3'):
    model.eval()
    extractor = RegionExtractor(image, samples, opts)
    for i, regions in enumerate(extractor):
        if opts['use_gpu']:
            regions = regions.cuda()
        with torch.no_grad():
            feat = model(regions, out_layer=out_layer)
        if i==0:
            feats = feat.detach().clone()
        else:
            feats = torch.cat((feats, feat.detach().clone()), 0)
    return feats


In [44]:
def train(model, criterion, optimizer, pos_feats, neg_feats, maxiter, in_layer='fc4'):
    model.train()

    batch_pos = opts['batch_pos']
    batch_neg = opts['batch_neg']
    batch_test = opts['batch_test']
    batch_neg_cand = max(opts['batch_neg_cand'], batch_neg)

    pos_idx = np.random.permutation(pos_feats.size(0))
    neg_idx = np.random.permutation(neg_feats.size(0))
    while(len(pos_idx) < batch_pos * maxiter):
        pos_idx = np.concatenate([pos_idx, np.random.permutation(pos_feats.size(0))])
    while(len(neg_idx) < batch_neg_cand * maxiter):
        neg_idx = np.concatenate([neg_idx, np.random.permutation(neg_feats.size(0))])
    pos_pointer = 0
    neg_pointer = 0

    for i in range(maxiter):

        # select pos idx
        pos_next = pos_pointer + batch_pos
        pos_cur_idx = pos_idx[pos_pointer:pos_next]
        pos_cur_idx = pos_feats.new(pos_cur_idx).long()
        pos_pointer = pos_next

        # select neg idx
        neg_next = neg_pointer + batch_neg_cand
        neg_cur_idx = neg_idx[neg_pointer:neg_next]
        neg_cur_idx = neg_feats.new(neg_cur_idx).long()
        neg_pointer = neg_next

        # create batch
        batch_pos_feats = pos_feats[pos_cur_idx]
        batch_neg_feats = neg_feats[neg_cur_idx]

        # hard negative mining
        if batch_neg_cand > batch_neg:
            model.eval()
            for start in range(0, batch_neg_cand, batch_test):
                end = min(start + batch_test, batch_neg_cand)
                with torch.no_grad():
                    score = model(batch_neg_feats[start:end], in_layer=in_layer)
                if start==0:
                    neg_cand_score = score.detach()[:, 1].clone()
                else:
                    neg_cand_score = torch.cat((neg_cand_score, score.detach()[:, 1].clone()), 0)

            _, top_idx = neg_cand_score.topk(batch_neg)
            batch_neg_feats = batch_neg_feats[top_idx]
            model.train()

        # forward
        pos_score = model(batch_pos_feats, in_layer=in_layer)
        neg_score = model(batch_neg_feats, in_layer=in_layer)

        # optimize
        loss = criterion(pos_score, neg_score)
        model.zero_grad()
        loss.backward()
        if 'grad_clip' in opts:
            torch.nn.utils.clip_grad_norm_(model.parameters(), opts['grad_clip'])
        optimizer.step()


In [45]:
# img_list= ["/home/waue0920/Pathway1_1/ORIG/ch6/321.bmp",
# "/home/waue0920/Pathway1_1/ORIG/ch6/322.bmp",
# "/home/waue0920/Pathway1_1/ORIG/ch6/323.bmp",
# "/home/waue0920/Pathway1_1/ORIG/ch6/324.bmp",
# "/home/waue0920/Pathway1_1/ORIG/ch6/325.bmp"]

image_path = "/home/waue0920/Pathway1_1/ORIG/ch6"
tracklet_num = 10




In [78]:


def run_MDnet(image_num,init_bbox):
    target_bbox = np.array(init_bbox)
    #image_num=str(num).zfill(4)    
    image_name = os.path.join(image_path,str(image_num)+".bmp")
    model = MDNet(opts['model_path'])
    model = model.cuda()

    criterion = BCELoss()
    model.set_learnable_params(opts['ft_layers'])
    init_optimizer = set_optimizer(model, opts['lr_init'], opts['lr_mult'])
    update_optimizer = set_optimizer(model, opts['lr_update'], opts['lr_mult'])

    tic = time.time()
    # Load first image
    image = Image.open(image_name).convert('RGB')

    # Draw pos/neg samples
    pos_examples = SampleGenerator('gaussian', image.size, opts['trans_pos'], opts['scale_pos'])(
                        target_bbox, opts['n_pos_init'], opts['overlap_pos_init'])

    neg_examples = np.concatenate([
                    SampleGenerator('uniform', image.size, opts['trans_neg_init'], opts['scale_neg_init'])(
                        target_bbox, int(opts['n_neg_init'] * 0.5), opts['overlap_neg_init']),
                    SampleGenerator('whole', image.size)(
                        target_bbox, int(opts['n_neg_init'] * 0.5), opts['overlap_neg_init'])])
    neg_examples = np.random.permutation(neg_examples)

    # Extract pos/neg features
    pos_feats = forward_samples(model, image, pos_examples)
    neg_feats = forward_samples(model, image, neg_examples)

    # Initial training
    train(model, criterion, init_optimizer, pos_feats, neg_feats, opts['maxiter_init'])
    del init_optimizer, neg_feats
    torch.cuda.empty_cache()

    # Train bbox regressor
    bbreg_examples = SampleGenerator('uniform', image.size, opts['trans_bbreg'], opts['scale_bbreg'], opts['aspect_bbreg'])(
                        target_bbox, opts['n_bbreg'], opts['overlap_bbreg'])
    bbreg_feats = forward_samples(model, image, bbreg_examples)
    bbreg = BBRegressor(image.size)
    bbreg.train(bbreg_feats, bbreg_examples, target_bbox)
    del bbreg_feats
    torch.cuda.empty_cache()

    # Init sample generators for update
    sample_generator = SampleGenerator('gaussian', image.size, opts['trans'], opts['scale'])
    pos_generator = SampleGenerator('gaussian', image.size, opts['trans_pos'], opts['scale_pos'])
    neg_generator = SampleGenerator('uniform', image.size, opts['trans_neg'], opts['scale_neg'])

    # Init pos/neg features for update
    neg_examples = neg_generator(target_bbox, opts['n_neg_update'], opts['overlap_neg_init'])
    neg_feats = forward_samples(model, image, neg_examples)
    pos_feats_all = [pos_feats]
    neg_feats_all = [neg_feats]

    spf_total = time.time() - tic

    for i in range(1, tracklet_num):
        tracklet_image_num = image_num+i
        tracklet_image_name = os.path.join(image_path,str(tracklet_image_num)+".bmp")
        tic = time.time()
        # Load image
        image = Image.open(image_name).convert('RGB')

        # Estimate target bbox
        samples = sample_generator(target_bbox, opts['n_samples'])
        sample_scores = forward_samples(model, image, samples, out_layer='fc6')

        top_scores, top_idx = sample_scores[:, 1].topk(5)
        top_idx = top_idx.cpu()
        target_score = top_scores.mean()
        target_bbox = samples[top_idx]
        if top_idx.shape[0] > 1:
            target_bbox = target_bbox.mean(axis=0)
        success = target_score > 0

        # Expand search area at failure
        if success:
            sample_generator.set_trans(opts['trans'])
        else:
            sample_generator.expand_trans(opts['trans_limit'])

        # Bbox regression
        if success:
            bbreg_samples = samples[top_idx]
            if top_idx.shape[0] == 1:
                bbreg_samples = bbreg_samples[None,:]
            bbreg_feats = forward_samples(model, image, bbreg_samples)
            bbreg_samples = bbreg.predict(bbreg_feats, bbreg_samples)
            bbreg_bbox = bbreg_samples.mean(axis=0)
        else:
            bbreg_bbox = target_bbox

        print(target_bbox)    
        # Save result
        #result[i] = target_bbox
        #result_bb[i] = bbreg_bbox
        
        # IOU
        if frameBBoxList[tracklet_image_num]:
            print("frame %d",tracklet_image_num)
            for idx,detect_bbox in enumerate(frameBBoxList[tracklet_image_num]):
                bboxD = detect_bbox.getRec()
                bboxT = target_bbox
                print(type(bboxD))
                print(type(bboxT))
                
                IOU_ratio = overlap_ratio(bboxD,bboxT)
                print(IOU_ratio)
                print("frame %d, bbox %d, ratio= %.3f",tracklet_image_num,idx,IOU_ratio)

        
        
        
        

        # Data collect
        if success:
            pos_examples = pos_generator(target_bbox, opts['n_pos_update'], opts['overlap_pos_update'])
            pos_feats = forward_samples(model, image, pos_examples)
            pos_feats_all.append(pos_feats)
            if len(pos_feats_all) > opts['n_frames_long']:
                del pos_feats_all[0]

            neg_examples = neg_generator(target_bbox, opts['n_neg_update'], opts['overlap_neg_update'])
            neg_feats = forward_samples(model, image, neg_examples)
            neg_feats_all.append(neg_feats)
            if len(neg_feats_all) > opts['n_frames_short']:
                del neg_feats_all[0]

        # Short term update
        if not success:
            nframes = min(opts['n_frames_short'], len(pos_feats_all))
            pos_data = torch.cat(pos_feats_all[-nframes:], 0)
            neg_data = torch.cat(neg_feats_all, 0)
            train(model, criterion, update_optimizer, pos_data, neg_data, opts['maxiter_update'])

        # Long term update
        elif i % opts['long_interval'] == 0:
            pos_data = torch.cat(pos_feats_all, 0)
            neg_data = torch.cat(neg_feats_all, 0)
            train(model, criterion, update_optimizer, pos_data, neg_data, opts['maxiter_update'])

        torch.cuda.empty_cache()
        spf = time.time() - tic
        spf_total += spf


In [36]:
init_box =[534,69,32,68]
run_MDnet(321,init_box)

[534.3164    67.58727   32.14178   68.301285]
[534.0089   68.88353  31.95863  67.91209]
[533.96924   68.01915   31.577648  67.10249 ]
[535.06824   68.60399   31.805538  67.58676 ]
[534.7778    69.17025   31.30214   66.517044]
[536.09485   68.1889    31.495996  66.92899 ]
[533.35974   69.20095   31.180506  66.258575]
[533.6759    69.56264   31.17722   66.251595]
[535.95294   70.45918   30.755682  65.35583 ]


In [80]:
for i in range(0,frame_num):
    
    if frameBBoxList[i]:
        print(i)
        init_bbox = frameBBoxList[i][0].getRec()
        print("init_bbox") 
        print(init_bbox)
        run_MDnet(i,init_box)

105
init_bbox
[ 10 234 248 239]
[ 17.611233 235.8674   246.9526   237.99063 ]
('frame %d', 106)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.73466782]
('frame %d, bbox %d, ratio= %.3f', 106, 0, array([0.73466782]))
[ 22.547249 239.15738  244.39986  235.53052 ]
('frame %d', 107)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.4138529]
('frame %d, bbox %d, ratio= %.3f', 107, 0, array([0.4138529]))
[ 18.023424 233.84167  244.6143   235.73718 ]
('frame %d', 108)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.5812227]
('frame %d, bbox %d, ratio= %.3f', 108, 0, array([0.5812227]))
[ 12.085611 242.57114  244.91006  236.02222 ]
('frame %d', 109)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.46671776]
('frame %d, bbox %d, ratio= %.3f', 109, 0, array([0.46671776]))
[ 19.068901 237.68327  242.11699  233.33052 ]
[  8.42114 239.5452  246.14969 237.21689]
[  5.3318114 244.62532   246.86118   237.90256  ]
[  9.15777 239.554   248.66582 239.6417 ]
('frame %d', 113)
<type 'numpy.ndarray'>
<ty

[ -0.49793395 226.35693    249.77951    240.71494   ]
('frame %d', 121)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 121, 0, array([0.]))
[ -1.237819 242.4202   252.07442  242.92656 ]
('frame %d', 122)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 122, 0, array([0.]))
114
init_bbox
[207 147 199 331]
[  2.2016633 233.13803   250.96353   241.856    ]
('frame %d', 115)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.04749514]
('frame %d, bbox %d, ratio= %.3f', 115, 0, array([0.04749514]))
[ -3.956688 222.35861  258.18945  248.81967 ]
('frame %d', 116)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.00644994]
('frame %d, bbox %d, ratio= %.3f', 116, 0, array([0.00644994]))
[ -1.836821 231.09042  253.02084  243.83865 ]
('frame %d', 117)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 117, 0, array([0.]))
[  6.468596 225.30688  256.21658  246.91843 ]
('frame %d', 118)
<type

[  4.2594194 229.57954   250.74771   241.64792  ]
('frame %d', 128)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 128, 0, array([0.]))
[ -1.8757324 227.2017    256.86383   247.54208  ]
('frame %d', 129)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 129, 0, array([0.]))
121
init_bbox
[312 141 153 338]
[  4.0319123 233.30606   243.56808   234.72891  ]
('frame %d', 122)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 122, 0, array([0.]))
[ -2.077736 234.45798  245.66553  236.75021 ]
[  2.9692993 230.86543   246.87383   237.9147   ]
('frame %d', 124)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 124, 0, array([0.]))
[  2.8128326 233.16464   246.67416   237.72229  ]
('frame %d', 125)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 125, 0, array([0.]))
[  1.4950073 242.84546   247.24345   238.2709  

[  5.059259 234.52682  247.48967  238.50818 ]
('frame %d', 129)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 129, 0, array([0.]))
[  4.6980057 234.08174   248.41626   239.40115  ]
('frame %d', 130)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 130, 0, array([0.]))
[ -1.5133682 226.93689   248.67041   239.64609  ]
('frame %d', 131)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 131, 0, array([0.]))
[  3.9932053 238.66531   246.61215   237.66252  ]
('frame %d', 132)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 132, 0, array([0.]))
[  6.7710466 228.02054   250.23584   241.15471  ]
('frame %d', 133)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 133, 0, array([0.]))
[  3.2022507 222.33554   252.64795   243.4793   ]
('frame %d', 134)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d

[  0.84906006 234.36377    247.11252    238.14473   ]
('frame %d', 135)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 135, 0, array([0.]))
[  3.0920777 237.09518   243.83842   234.98946  ]
('frame %d', 136)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 136, 0, array([0.]))
[ -9.046471 232.91528  244.30371  235.43784 ]
('frame %d', 137)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 137, 0, array([0.]))
[  0.57623136 243.1571     248.88054    239.84856   ]
('frame %d', 138)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 138, 0, array([0.]))
[  2.080197 234.58774  244.7061   235.8256  ]
('frame %d', 139)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 139, 0, array([0.]))
[  1.4284576 233.74292   246.55046   237.60303  ]
('frame %d', 140)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('fram

[  0.75951385 229.78238    258.18307    248.81355   ]
('frame %d', 141)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 141, 0, array([0.]))
[  0.9471985 231.43747   259.9169    250.48447  ]
('frame %d', 142)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 142, 0, array([0.]))
[  0.97034454 228.34946    260.95386    251.48381   ]
('frame %d', 143)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 143, 0, array([0.]))
[ -0.9982559 232.6865    259.233     249.82541  ]
('frame %d', 144)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 144, 0, array([0.]))
[  3.143312 229.15022  261.63022  252.13565 ]
('frame %d', 145)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('frame %d, bbox %d, ratio= %.3f', 145, 0, array([0.]))
[ -2.3380141 220.81625   266.76773   257.08673  ]
('frame %d', 146)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[0.]
('

KeyboardInterrupt: 

In [74]:
a = np.array([534,69,32,68])
a.ndim

1